## MM

In [0]:
from gensim.models.word2vec import FAST_VERSION
FAST_VERSION

In [0]:
from IPython.display import clear_output
import os
import numpy as np
!pip install nodevectors
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [0]:
# fix seeds
s = 42
from numpy.random import seed
seed(s)

## Read and preprocess data

In [0]:
edges = pd.read_csv('data/edges.csv')
ids = pd.read_csv('data/ids.csv')
vertices = pd.read_csv('data/vertices.csv')

In [0]:
edges_wo_ids = edges[(~edges['id_1'].isin(ids['id'])) & (~edges['id_2'].isin(ids['id']))]
edges_wo_ids.rename({'n_transactions': 'weight'}, axis=1, inplace=True)

## Create graph

In [0]:
import random
from random import shuffle, sample
import tqdm
import networkx as nx

In [0]:
%%time
graph = nx.from_pandas_edgelist(edges_wo_ids, "id_1", "id_2", "weight")
graph = graph.to_undirected()

CPU times: user 20.3 s, sys: 1.24 s, total: 21.5 s
Wall time: 21.5 s


## Fit node2vec
### We fit to a graph with some edges dropped to later use them for boosting training

In [0]:
# github.com/VHRanger/nodevectors
from nodevectors import Node2Vec
import multiprocessing as mp
import pickle

In [0]:
%%time
workers = mp.cpu_count()

if True:
    n2v = Node2Vec(
        threads=workers,
        n_components=256,
        walklen=10, 
        epochs=20,
    )
    n2v.fit(graph)
    n2v.save_vectors('n2v256.bin')

Making walks... Done, T=88.95
Mapping Walk Names... 

In [0]:
!zip n2v256.zip n2v256.bin